In [146]:
# Librerias ...
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn import linear_model
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn import ensemble
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model.ridge import Ridge
from sklearn.linear_model import Lasso
import warnings
import math
from sklearn.metrics import fbeta_score, make_scorer

%matplotlib inline

In [147]:
# Lectura de datos archivo CSV
df = pd.read_csv('C:\\JSONFINAL\\salida.csv', header=0, index_col=0)

## Data Cleaning & Check Data

Aqui partiendo del Jupyter Notebook 0.Analisis_Exploratorio_Datos vamos aplicar algunas conclusiones tanto de limpieza como de validacion de datos, para asi detectar valores anómalos.

In [148]:
# Eliminacion de filas repetidas
df = df.groupby(['Caso X', 'D','L','Beta','Velocidad','Angulo','A','B','Q', 'Reynolds','TotalAreaInlet', 'P']).size().reset_index()

# Eliminacion de filas que tienen un coeficiente de Reynols muy negativo. Posiblemente a simulaciones que han producido un error.
# xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

# Eliminacion de columnas A y B, por ser variables que son de salida de la simulacion
X = df.drop(['A','B'], axis=1, inplace=True)

# Eliminacion de la variable categorica Caso X, al ser una variable que no aporta nada, ya que tan solo define dado un caso
# a un conjunto de caracteristicas geometricas D,L,Beta
X = df.drop(['Caso X'],axis=1, inplace=True)

# Eliminacion de la columna Reynolds, al saber que existe una relacion y Q
X = df.drop(['Reynolds'], axis=1, inplace=True)

# Eliminacion de la columna TotalAreaInlet, al saber que existe una relacion y Q
X = df.drop(['TotalAreaInlet'], axis=1, inplace=True)

# Eliminacion de la columna Velocidad, al saber que existe una relacion y Q
X = df.drop(['Velocidad'], axis=1, inplace=True)

# Featuring Engineering
df['Q'] = abs(df['Q'])
df['QQ'] = df['Q'] * df['Q']
df['QPlusQQ'] = df['Q'] + (df['Q'] * df['Q'])

# Por supuesto quitamos como variable de entrada la variable a PREDECIR, pero no inplace porque luego se lo asignaremos a Y
X = df.drop(['P'], axis=1).values


In [79]:
df

,D,L,Beta,Angulo,Q,P,0
0,0.040,0.9,0.2,0,0.000064,19.252066,1
1,0.040,0.9,0.2,170,0.000064,24.926916,1
2,0.040,0.9,0.2,-145,0.000064,82.646603,1
3,0.040,0.9,0.2,-170,0.000064,30.105155,1
4,0.040,0.9,0.2,-120,0.000064,116.412337,1
5,0.040,0.9,0.2,145,0.000064,82.088580,2
6,0.040,0.9,0.2,120,0.000067,119.307336,1
7,0.040,0.9,0.2,0,0.000085,27.382207,1
8,0.040,0.9,0.2,170,0.000085,40.487631,1
9,0.040,0.9,0.2,-145,0.000085,128.531086,1


In [85]:
# Definimos nuestra variable OBJETIVO a predecir "Diferencia de Presion"
y = df['P'].values

## Buscando el modelo
Ahora vamos a buscar el modelo que mejor se ajuste a nuestro datos.

Primero que nada vamos a dividir en TRAIN / VALIDATION para buscar el MODELO que mejor parezca que concuerde y finalmente un conjunto de TEST para ver como ese candidato modelo predice sobre un conjunto de datos que nunca vio.

In [144]:
# División entre set de TRAIN+VALIDATION y TEST
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=.2)

print('X: ',X.shape,' y:', y.shape)
print('X_train_val: ',X_train_val.shape, ' y_train_val: ',y_train_val.shape)
print('X_test: ',X_test.shape, ' y_test: ',y_test.shape)

('X: ', (450L, 8L), ' y:', (450L,))
('X_train_val: ', (360L, 8L), ' y_train_val: ', (360L,))
('X_test: ', (90L, 8L), ' y_test: ', (90L,))


Como se puede apreciar aqui el mayor problema es que no disponemos de muchas filas ... asi que "si o si" debemos realizar CROSS VALIDATION.

In [162]:
# creamos las variables de score para guardar cada una de las ejecuciones
score_lm = []
score_rdg = []
score_lss = []
score_knn = []
score_dt = []
score_rf = []
score_gb = []

# Al disponer de pocas filas realizamos CROSS Validation 
# 10-Fold Cross-Validation            
kf = KFold(n_splits = 10, shuffle=True)

for k, (train, val)  in enumerate (kf.split(X_train_val)):

    X_train, y_train, X_val, y_val = X[train], y[train], X[val], y[val]
    
    
    # Linear Regression 
    lm = LinearRegression().fit(X_train, y_train)
    
    # Calculamos SCORE
    score_training = lm.score(X_train, y_train)
    score_val = lm.score(X_val, y_val)
    
    print("[fold {0}] Linear Regression: score in TRAINING: {1:.2f} in TEST {2:.2f}".format(k, score_training, score_val))  
    
    # Guardamos el SCOREque nos ha dado en validacion para luego poder hacer la media
    score_lm.append(score_val)
    
    
    
    # Ridge Regression 
    rdg = Ridge().fit(X_train, y_train)
    
    # Calculamos SCORE
    score_training = rdg.score(X_train, y_train)
    score_val = rdg.score(X_val, y_val)
    
    print("[fold {0}] Ridge Regression: score in TRAINING: {1:.2f} in TEST {2:.2f}".format(k, score_training, score_val))  
  
    # Guardamos el SCOREque nos ha dado en validacion para luego poder hacer la media
    score_rdg.append(score_val)
    
    
    
    # Lasso Regression 
    lss = Lasso().fit(X_train, y_train)
    
     # Calculamos SCORE
    score_training = lss.score(X_train, y_train)
    score_val = lss.score(X_val, y_val)
    
    print("[fold {0}] Lasso Regression score in TRAINING: {1:.2f} in TEST {2:.2f}".format(k, score_training, score_val))  
     
    score_lss.append(score_val)
    
    
    
    # KNeighborsRegressor Regression 
    knn = KNeighborsRegressor().fit(X_train, y_train)
    
     # Calculamos SCORE
    score_training = knn.score(X_train, y_train)
    score_val = knn.score(X_val, y_val)
        
    print("[fold {0}] KNeighborsRegressor score in TRAINING: {1:.2f} in TEST {2:.2f}".format(k, score_training, score_val))  
        
    score_knn.append(score_val)
    
    
    
    # Arbol de decision        
    dt = tree.DecisionTreeRegressor().fit(X_train, y_train)
    
    # Calculamos SCORE
    score_training = dt.score(X_train, y_train)
    score_val = dt.score(X_val, y_val)
    
    print("[fold {0}] DecisionTreeRegressor score in TRAINING: {1:.2f} in TEST {2:.2f}".format(k, score_training, score_val))  
       
    score_dt.append(score_val)
    
    
    
    # Random Forest
    rf = ensemble.RandomForestRegressor().fit(X_train, y_train)
    
    # Calculamos SCORE
    score_training = rf.score(X_train, y_train)
    score_val = rf.score(X_val, y_val)
    
    print("[fold {0}] RandomForestRegressor score in TRAINING: {1:.2f} in TEST {2:.2f}".format(k, score_training, score_val))  

    score_rf.append(score_val)
    
    
    # Gradient Boosting
    gb = ensemble.GradientBoostingRegressor().fit(X_train, y_train)
    
     # Calculamos SCORE
    score_training = gb.score(X_train, y_train)
    score_val = gb.score(X_val, y_val)
 
    print("[fold {0}] GradientBoostingRegressor score in TRAINING: {1:.2f} in TEST {2:.2f}".format(k, score_training, score_val))  

    score_gb.append(score_val)
          
        
# Una vez ya entrenados los distintos modelos y realizado el CROSS VALIDATION, mostramos las medias de sus SCORES
print('\n')

print('Linear Regression avg(SCORE):',np.mean(score_lm))
print('Ridge Regression avg(SCORE):',np.mean(score_rdg))
print('Lasso Regression avg(SCORE):',np.mean(score_lss))
print('KNeighbors Regression avg(SCORE): ',np.mean(score_knn))
print('Decission Tree avg(SCORE)',np.mean(score_dt))
print('Random Forest avg(SCORE)',np.mean(score_rf))
print('Gradient Boosting avg(SCORE)',np.mean(score_gb))

[fold 0] Linear Regression: score in TRAINING: 0.24 in TEST -0.10
[fold 0] Ridge Regression: score in TRAINING: 0.01 in TEST -0.20
[fold 0] Lasso Regression score in TRAINING: 0.01 in TEST -0.18
[fold 0] KNeighborsRegressor score in TRAINING: 0.76 in TEST 0.69
[fold 0] DecisionTreeRegressor score in TRAINING: 1.00 in TEST 0.97
[fold 0] RandomForestRegressor score in TRAINING: 0.99 in TEST 0.97
[fold 0] GradientBoostingRegressor score in TRAINING: 0.99 in TEST 0.98
[fold 1] Linear Regression: score in TRAINING: 0.23 in TEST 0.17
[fold 1] Ridge Regression: score in TRAINING: 0.01 in TEST -0.02
[fold 1] Lasso Regression score in TRAINING: 0.01 in TEST -0.02
[fold 1] KNeighborsRegressor score in TRAINING: 0.77 in TEST 0.59
[fold 1] DecisionTreeRegressor score in TRAINING: 1.00 in TEST 0.94
[fold 1] RandomForestRegressor score in TRAINING: 1.00 in TEST 0.95
[fold 1] GradientBoostingRegressor score in TRAINING: 0.99 in TEST 0.95
[fold 2] Linear Regression: score in TRAINING: 0.23 in TEST 0.2

De los scores anteriores ya vemos que ningun modelo lineal es representativo de los datos que estamos buscando  y que necesitamos trabjar con modelos mas complejos.

Nos vamos por tanto a decantar a buscar hyperparametros tanto para Decission Tree, Random Forest como Gradient Boosting.

A priori Gradient Boosting parece un pelin mejor pero ... también hay que tener en cuenta que computacionalmente requiere más tiempo para realizar el entrenamiento.

In [157]:
# División entre set de TRAIN+VALIDATION y TEST
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=.2)

parametros_modelo = {
    'max_depth': list(range(3, 10 + 1)),
    'max_features': [None, 'log2', 'sqrt'],
}


# DecisionTreeRegressor        
dt = tree.DecisionTreeRegressor()
grid_search = GridSearchCV(dt, parametros_modelo,  cv = 10).fit(X_train, y_train)

# Mejores parametros para el modelo
print("DecisionTreeRegressor: {0:.2f} params  {1}".format(grid_search.best_score_, grid_search.best_params_))


parametros_modelo = {
    'n_estimators': [10, 20, 50, 100, 200, 500, 1000],
    'max_depth': list(range(3, 10 + 1)),
    'max_features': [None, 'log2', 'sqrt'],
}

# RandomForestRegressor
rf = ensemble.RandomForestRegressor()
grid_search = GridSearchCV(rf, parametros_modelo,  cv = 10).fit(X_train, y_train)

# Mejores parametros para el modelo
print("RandomForestRegressor: {0:.2f} params  {1}".format(grid_search.best_score_, grid_search.best_params_))
      

parametros_modelo = {
    'n_estimators': [10, 20, 50, 100, 200, 500, 1000],
    'max_depth': list(range(3, 10 + 1)),
    'max_features': [None, 'log2', 'sqrt'],
}

# GradientBoostingRegressor
gb = ensemble.GradientBoostingRegressor()
grid_search = GridSearchCV(rf, parametros_modelo,  cv = 10).fit(X_train, y_train)

# Mejores parametros para el modelo
print("GradientBoostingRegressor: {0:.2f} params  {1}".format(grid_search.best_score_, grid_search.best_params_))


RandomForestRegressor: 0.95 params  {}
RandomForestRegressor: 0.97 params  {'max_features': None, 'n_estimators': 10, 'max_depth': 7}
RandomForestRegressor: 0.97 params  {'max_features': None, 'n_estimators': 1000, 'max_depth': 8}


Finalmente ya vemos que el mejor modelo es el GradientBoostingRegressor y que sus parámetros más óptimos para los que ha sido entrenado el modelo es:
    
Ahora vamos a ejecutar este modelo, para el conjunto de TEST y ya podremos ver con certeza cual es realmente el SCORE arrojado.
    

In [161]:
# Calculamos el R², el MSE y el MAE.

reg = ensemble.RandomForestRegressor(
    max_depth=7, n_estimators=10
).fit(X_train_val, y_train_val)

pred = reg.predict(X_test)
(
    reg.score(X_test, y_test), # R²
    metrics.mean_squared_error(y_test, pred), # MSE
    metrics.mean_absolute_error(y_test, pred) # MAE
)

(0.9791432491231095, 148.2732078320623, 8.557495533206053)